### Baseline Model

In [1]:
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras import models, layers, optimizers

from sklearn.model_selection import train_test_split

import pandas as pd
import numpy as np

import os
import matplotlib.pyplot as plt
import seaborn as sns
from numpy import random

In [10]:
from google.colab import drive 
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [13]:
!pwd

/content


In [15]:
!ls

gdrive	sample_data


In [16]:
!cd gdrive

In [19]:
main = os.listdir("gdrive/smArt/wikiart")

genres = []
for genre in main:
    if genre[0] != ".":
        genres.append(genre)

data = sorted(genres)

FileNotFoundError: ignored

In [ ]:
poc_list = ["Realism", "Impressionism"]

In [ ]:
from PIL import Image
import glob, os

paintings_list = []
size = 128, 128

for index, genre in enumerate(poc_list):
    print(f"the genre is {genre}")
    g = os.listdir("../raw_data/wikiart/" + str(genre))
    
    for i, painting in enumerate(g):
        with Image.open("../raw_data/wikiart/" + genre + "/" +painting) as im:
            img_resized = im.resize(size)
            image_array = np.array(img_resized)
            #print(image_array.shape)
            #paintings_list.append(image_array)
            string = g[i][:-4]
            string = string.replace("-"," ")
            string = string.split("_")
            string.insert(0, genre)
            string.append(image_array)
            paintings_list.append(string)
    
            if i % 100 == 0:
                print(i+1)

the genre is Realism
1
101
201
301
401
501
601
701
801
901
1001
1101
1201
1301
1401
1501
1601
1701
1801
1901
2001
2101
2201
2301
2401
2501
2601
2701
2801
2901
3001
3101
3201
3301
3401
3501
3601
3701


In [ ]:
df = pd.DataFrame(paintings_list)
df

In [ ]:
def load_model():
    
    # $CHALLENGIFY_BEGIN
    
    model = VGG16(weights="imagenet", include_top=False, input_shape=(128,128,3))
    
    # $CHALLENGIFY_END
    
    return model

In [ ]:
def set_nontrainable_layers(model):
    
    # $CHALLENGIFY_BEGIN
    # Set the first layers to be untrainable
    model.trainable = False
    
    # $CHALLENGIFY_END
    
    return model

In [ ]:
def add_last_layers(model):
    '''Take a pre-trained model, set its parameters as non-trainable, and add additional trainable layers on top'''
    # $CHALLENGIFY_BEGIN
    base_model = set_nontrainable_layers(model)
    flatten_layer = layers.Flatten()
    dense_layer = layers.Dense(500, activation='relu')
    prediction_layer = layers.Dense(1, activation='sigmoid')
    
    
    model = models.Sequential([
        base_model,
        flatten_layer,
        dense_layer,
        prediction_layer
    ])
    # $CHALLENGIFY_END
    return model


In [ ]:
def build_model():
    # $CHALLENGIFY_BEGIN    
    model = load_model()
    model = add_last_layers(model)
    
    opt = optimizers.Adam(learning_rate=1e-4)
    model.compile(loss='binary_crossentropy',
                  optimizer=opt,
                  metrics=['accuracy'])
    return model
    # $CHALLENGIFY_END

In [ ]:
def tolist(x):
    return x.tolist()

In [ ]:
X = df[3]

In [ ]:
np.asarray(X).astype(np.float32)

In [ ]:
#turn genre in binary
def genre_to_binary(x):
    if x == "Realism":
        return 0
    if x == "Impressionism":
        return 1

In [ ]:
df["Impressionism"] = df[0].apply(genre_to_binary)

In [ ]:
y = df[0]

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [ ]:
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train)

In [ ]:
X_train.

In [ ]:
[print(i.shape, i.dtype) for i in model.inputs]
[print(o.shape, o.dtype) for o in model.outputs]
[print(l.name, l.input_shape, l.dtype) for l in model.layers]

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping

model = build_model()

es = EarlyStopping(monitor = 'val_accuracy', 
                   mode = 'max', 
                   patience = 5, 
                   verbose = 1, 
                   restore_best_weights = True)

history = model.fit(X_train, y_train, 
                    validation_data=(X_val, y_val), 
                    epochs=50, 
                    batch_size=16, 
                    callbacks=[es])